# Set up

In [1]:
import os
from google import genai
from google.genai import types
from dotenv import load_dotenv
load_dotenv()
client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))

In [48]:
client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))
myfile = client.files.upload(file='Telesales.mp3')

# Describe Audio direct

In [30]:
response = client.models.generate_content(
  model='gemini-2.0-flash',
  contents=[
    'Describe this audio clip in Thai and differentiate between the Telesales and the Customer.',
    myfile,
  ]
)

print(response.text)

แน่นอนค่ะ นี่คือคำอธิบายเสียงในภาษาไทย พร้อมกับการเปรียบเทียบระหว่าง Telesales กับลูกค้า:

**คำอธิบายเสียง:**
คลิปเสียงนี้เป็นการสนทนาระหว่างพนักงานขายทางโทรศัพท์ (Telesales) และลูกค้า บทสนทนาเริ่มต้นด้วยการแนะนำตัวของพนักงานขายจากบริษัทประกันชีวิต และการเสนอโครงการพิเศษชื่อ "คุ้มดีมีคืน" ซึ่งเป็นแผนออมทรัพย์ในรูปแบบประกันชีวิตสะสมทรัพย์ พนักงานขายอธิบายรายละเอียดผลประโยชน์และข้อดีต่างๆ ของโครงการ รวมถึงผลตอบแทนที่ลูกค้าจะได้รับ และการลดหย่อนภาษี ลูกค้าแสดงความสนใจและสอบถามเกี่ยวกับความเสี่ยงในการลงทุน พนักงานขายอธิบายว่าไม่ใช่การลงทุน แต่เป็นการออมเงินที่มาพร้อมความคุ้มครองชีวิต จากนั้นลูกค้าตกลงเข้าร่วมโครงการ และพนักงานขายยืนยันข้อมูลส่วนตัวของลูกค้า รวมถึงแจ้งรายละเอียดการจัดส่งเอกสาร

**การเปรียบเทียบ Telesales กับลูกค้า:**
*   **Telesales:**
    *   พูดจาสุภาพ อ่อนน้อม และเป็นกันเอง
    *   มีน้ำเสียงที่ชัดเจน มั่นใจ และน่าเชื่อถือ
    *   มีความรู้และความเข้าใจในผลิตภัณฑ์อย่างละเอียด
    *   สามารถอธิบายรายละเอียดต่างๆ ได้อย่างชัดเจนและเข้าใจง่าย
    *   สามารถตอบคำถามและข้อสงสั

# Transcribe audio

In [ ]:
response = client.models.generate_content(
  model='gemini-2.0-flash',
  contents=[
    'Transcribe this audio clip in Thai and differentiate between the Telesales and the Customer.',
    myfile,
  ]
)

print(response.text)

# QA feature

In [4]:
from typing import List, Dict, Any, Union, Optional, Tuple, Literal
from pydantic import BaseModel, Field

In [25]:
from pydantic import BaseModel, Field
from typing import List, Literal

class Introduction(BaseModel):
    greeting: str = Field(description='Greeting of the person in the conversation')
    name: str = Field(description='Name of the person in the conversation')
    company_affiliation: str = Field(description='Company affiliation of the telesales in the conversation')
    license_number: str = Field(description='License number of the telesales in the conversation')  
    permission_record_audio: bool = Field(description='Permission to record the audio of the conversation')

class Person(BaseModel):
    position: Literal['Telesales', 'Customer'] = Field(description='Position of the person in the conversation')
    name: str = Field(description='Name of the person in the conversation')
    mannered_voice_tone: str = Field(description='Mannered voice tone of the person in the conversation')  

class RegulatoryCompliance(BaseModel):
    proper_introduction: Introduction = Field(description='Proper introduction of the telesales in the conversation') 
    # mannered_voice_tone: List[Person] = Field(description='Mannered voice tone of the people in the conversation')  
    # prohibited_words: List[str] = Field(description="""Prohibited phrases or words in the conversation (in Thai) (only explicit words) topics:)
    #                                     การให้คำมั่นสัญญาที่เกินจริง
    #                                     คำที่สร้างความเข้าใจผิดหรือชี้นำผิด
    #                                     การอ้างหน่วยงานหรือบุคคล
    #                                     การใช้คำที่กดดัน
    #                                     วลีที่ละเมิดสิทธิส่วนบุคคลหรือข้อมูลส่วนบุคคล""")

class checkprohibited_words(BaseModel):
    prohibited_words: List[str] = Field(description="""Prohibited phrases or words in the conversation (in Thai) (only explicit words) topics:
                                        คำหยาบคาย หรือคำที่เกี่ยวข้องกับเรื่องทางเพศ
                                        การให้คำมั่นสัญญาที่เกินจริง "ไม่เสี่ยงแน่นอน"
                                        การอ้างหน่วยงานหรือบุคคล
                                        การใช้คำที่กดดัน
                                        วลีที่ละเมิดสิทธิส่วนบุคคลหรือข้อมูลส่วนบุคคล""")



In [16]:
call_chat = """"**Telesales:** สวัสดีค่ะ ขอเรียนสายคุณโชคดีมีเงินค่ะ ดิฉันนางสาวขายดีทวีทรัพย์ติดต่อจากบริษัทคุ้มดีจำกัดในเครือธนาคาร Fin for you ทางบริษัทได้จัดทำโครงการพิเศษชื่อว่าคุ้มดีมีคืน สะดวกเรียนสายได้ซักครู่ไหมคะ

**Customer:** สะดวกค่ะ

**Telesales:** ขอบพระคุณค่ะ หนูขออนุญาตบันทึกสายสนทนานะคะ ดิฉัน นางสาวขายดีทวีทรัพย์ เลขที่ใบอนุญาตตัวแทน ตท.1902/2025 ค่ะ

ทางโครงการขอเรียนเชิญคุณลูกค้าเข้าร่วมโครงการคุ้มดีมีคืนค่ะ ซึ่งเป็นโครงการออมทรัพย์พิเศษในรูปแบบการประกันชีวิตสะสมทรัพย์ ซึ่งผลประโยชน์ที่ลูกค้าจะได้รับแตกต่างจากที่อื่นๆ เพราะการันตีผลตอบแทนรวมตลอดสัญญาสูงสุดถึง 616 เปอร์เซ็นต์ของทุนประกันชีวิตเลยค่ะ โดยคุณลูกค้าไม่ต้องใช้เงินก้อนเป็นหมื่นเป็นแสนเลยนะคะ เพียงออมเงินทุกๆเดือนเท่าๆกัน ตัวอย่างเช่นคุณลูกค้าจ่ายเบี้ยเพียงวันละ 90 บาท หรือรายเดือนเดือนละ 2,700 บาท คุณลูกค้าจะได้เงินคืนทุกปีตั้งแต่ปีที่ 1 จนถึงปีที่ 14 การันตีรับเงินคืนปีละ 3,200 บาท 14 14 ปี เท่ากับได้ออมเพิ่ม เพิ่มขึ้น 44,800 บาท สิ้นสิ้นปีที่ 15-24 รับเงินการันตีเพิ่มขึ้นไปอีก เป็นปีละ 6,400 บาท รวม 10 ปีเป็นเงิน เป็นเงินเพิ่ม 64,000 บาทค่ะ และเมื่อสิ้นปีที่ 24 ที่ครบอายุสัญญา คุณลูกค้าจะได้เงินออม 384,000 บาท เมื่อรวมกับเงินคืนที่ได้ไปในปีที่ 1 ถึงปีที่ 24 แล้วเป็นเงิน 492,800 บาทเลยทีเดียวค่ะ ที่สำคัญ คุณลูกค้าออมเงินเพียงวันละ เพียงแค่ 14 ปีแรก แต่มีเงินเพิ่มทุกๆปีถึง 24 ปีเลยค่ะ นอกจากนี้ ลูกค้ายังสามารถนำยอดเบี้ยที่ชำระในปีที่ 14 ปีแรกไปลดหย่อนภาษีในแต่ละปีและยังได้รับความคุ้มครองกรณีลูกค้าสูญเสียชีวิตระหว่างปีที่ 24 นี้ในวงเงินที่สูงถึง 384,000 บาท ตั้งแต่วันที่คุณลูกค้าได้รับการอนุมัติกรมธรรม์ให้อีกด้วย ซึ่งเงินก้อนนี้สามารถส่งต่อเป็นมรดกให้ลูกหลานหรือพ่อแม่ได้ค่ะ โดยคุ้มครองทุกกรณีของการเสียชีวิต ยกเว้นการฆ่าตัวตายปีแรกหรือถูกผู้รับผลประโยชน์ฆาตกรรมค่ะ โครงการนี้เป็นประกันชีวิตแบบสะสมทรัพย์ ที่ลูกค้าได้รับผลตอบแทนครบทุกด้าน ทั้งผลตอบแทนในรูปแบบเงินคืนการันตีทุกปี 24 ปี ความคุ้มครองชีวิตและประโยชน์ในการลดหย่อนภาษี เราจึงจัดทำโครงการนี้เป็นพิเศษไม่เหมือนใครเพื่อคุณลูกค้าโดยเฉพาะเลยค่ะ คุณลูกค้า สนใจเข้าร่วมโครงการหรือไม่คะ

**Customer:** แล้ว แล้วมันเอาไปลงทุน ทุนยังไงอ่ะคะ แล้วมีความเสี่ยงไหมคะ

**Telesales:** ขายดีขอแจ้งคุณลูกค้าแบบนี้นะค่ะ โครงการนี้ไม่ได้เป็นการลงทุนค่ะคุณลูกค้า แต่เป็นการออมเงินและได้สิทธิประโยชน์ในการคุ้มครองชีวิตไปด้วย ซึ่งต่างจากการนำเงินไปลงทุนในหุ้นหรือตลาดทุนที่คุณลูกค้าอาจจะเสี่ยงที่จะขาดทุนหรือสูญเสียเงินจากการลงทุนได้ค่ะ

**Customer:** อ๋อๆ เข้าใจแล้วค่ะ

**Telesales:** คุณลูกค้ายืนยัน เข้า ร่วมโครงการนะคะ ขายดีขอทวนชื่อคุณลูกค้า เป็นคุณโชคดีมีเงินอยู่บ้านเลขที่ 69/96 หมู่บ้านโชคดีทวีสุข ถนนรัชดาภิเษก แขวงดินแดง เขตดินแดง กรุงเทพ 10310 ถูกต้องไหมคะ

**Customer:** ถูกต้องค่ะ

**Telesales:** ขอบคุณค่ะคุณลูกค้า ภายใน 15 วันทำการทางบริษัทจะส่งเอกสารรายละเอียดการออมให้ ให้เป็นให้เก็บไว้เป็นหลักฐาน โดยเป็นกรมธรรม์ฉบับสะสมทรัพย์ ออมเพิ่มสุข ที่จะระบุผลตอบแทนแบบการันตีทุกๆตัวเลขตามที่เจ้าหน้าที่แจ้งทั้งหมด จัดส่งตามที่ ที่อยู่ที่ระบุไว้เลยไหมคะคุณลูกค้า

**Customer:** ได้เลยค่ะ

**Telesales:** ขอบคุณค่ะคุณลูกค้า ขายดีขอขอบพระคุณคุณลูกค้าเป็นอย่างสูง และขอให้คุณลูกค้ามีสุขภาพแข็งแรงตลอดปี 2525 นะคะ สวัสดีค่ะ

**Customer:** สวัสดีค่ะ"""

In [ ]:
import json
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import List, Literal

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0 ,api_key=os.getenv("GEMINI_API_KEY"))


parser = PydanticOutputParser(pydantic_object=RegulatoryCompliance)


prompt = PromptTemplate(
    template="""
    คุณเป็นระบบตรวจสอบคุณภาพของสายโทรศัพท์
    ต่อไปนี้คือบทสนทนา:
    ```text
    {conversation}
    ```
    โปรดวิเคราะห์และสรุปข้อมูลตามโครงสร้าง JSON ด้านล่าง:
    {format_instructions}
    """,
    input_variables=["conversation"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | llm | parser
structured_output = chain.invoke({"conversation": call_chat})



In [13]:
structured_output.dict()

C:\Users\sirip\AppData\Local\Temp\ipykernel_30504\656136847.py:1: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  structured_output.dict()


{'proper_introduction': {'greeting': 'สวัสดีค่ะ ขอเรียนสายคุณโชคดีมีเงินค่ะ',
  'name': 'นางสาวขายดีทวีทรัพย์',
  'company_affiliation': 'บริษัทคุ้มดีจำกัดในเครือธนาคาร Fin for you',
  'license_number': 'ตท.1902/2025',
  'permission_record_audio': True}}

In [26]:
paser_prohibited_words = PydanticOutputParser(pydantic_object=checkprohibited_words)

prompt = PromptTemplate(
    template="""
    คุณเป็นระบบตรวจสอบคุณภาพของสายโทรศัพท์
    ต่อไปนี้คือบทสนทนา:
    ```text
    {conversation}
    ```
    โปรดวิเคราะห์และสรุปข้อมูลตามโครงสร้าง JSON ด้านล่าง:
    {format_instructions}
    """,
    input_variables=["conversation"],
    partial_variables={"format_instructions": paser_prohibited_words.get_format_instructions()},
)

chain = prompt | llm | paser_prohibited_words
structured_output = chain.invoke({"conversation": call_chat})

In [27]:
structured_output.dict()

C:\Users\sirip\AppData\Local\Temp\ipykernel_30504\656136847.py:1: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  structured_output.dict()


{'prohibited_words': []}

# Describe from Transcribe

In [ ]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import PydanticOutputParser

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0, api_key=os.getenv("GEMINI_API_KEY"))

prompt = PromptTemplate(
    template="""
You are an AI assistant that helps summarize product features from conversations. in thai.
Here is a conversation about the product:
```text
{conversation}
```
""",
)

chain = prompt | llm 


In [ ]:
structured_output = chain.invoke({"conversation": call_chat})

In [47]:
print(structured_output.content)

สรุปคุณสมบัติของผลิตภัณฑ์ "คุ้มดีมีคืน" จากบทสนทนา:

*   **ประเภทผลิตภัณฑ์:** ประกันชีวิตสะสมทรัพย์ ไม่ใช่การลงทุน
*   **ผลตอบแทน:** การันตีผลตอบแทนรวมสูงสุด 616% ของทุนประกันชีวิต
*   **การออม:** ออมเงินรายเดือน (ตัวอย่าง: วันละ 90 บาท หรือเดือนละ 2,700 บาท)
*   **เงินคืน:**
    *   ปีที่ 1-14: รับเงินคืนปีละ 3,200 บาท
    *   ปีที่ 15-24: รับเงินคืนปีละ 6,400 บาท
    *   สิ้นปีที่ 24 (ครบอายุสัญญา): ได้รับเงินออม 384,000 บาท
    *   รวมเงินคืนตลอดสัญญา: 492,800 บาท
*   **ระยะเวลาการออม:** ออม 14 ปี แต่ได้รับเงินคืน 24 ปี
*   **ความคุ้มครองชีวิต:** คุ้มครองกรณีเสียชีวิต (ยกเว้นฆ่าตัวตายปีแรกหรือถูกผู้รับผลประโยชน์ฆาตกรรม) วงเงิน 384,000 บาท
*   **ประโยชน์ทางภาษี:** สามารถนำเบี้ยประกันที่ชำระในช่วง 14 ปีแรกไปลดหย่อนภาษีได้
*   **มรดก:** เงินก้อนสามารถส่งต่อเป็นมรดกให้ลูกหลานหรือพ่อแม่ได้
*   **เอกสาร:** บริษัทจะส่งเอกสารรายละเอียดกรมธรรม์ฉบับสะสมทรัพย์ "ออมเพิ่มสุข" ภายใน 15 วันทำการ
